# Generación aleatoria de instancias pequeñas

En este notebook se generaran 5 instancias pequeñas de JRP. El objetivo es usarlas en el notebook ./experimento, sobre el que se probara el aLgoritmo QAOA haciendo uso de computo del servicio de GPU que proporciona qiskit. Este servicio es un bridge hacia el framework cuQuantum de NVIDIA.

Para que la creación de instancias aleatorias sea realista con casos de uso industriales, se utilizaran las heurísticas propuestas por Alejandro.

* Las prioridades son números enteros entre 1 y 5, normalmente hay más cercanas al 5 que al 1.
* Las afinidades varían de manera continua entre 0 y 0.5.
* El numero de empleados suele ir de 3 a 20.
* El numero de vacantes suele ir de 5 a 30.

In [6]:
import random as r
import json

def create_random_instance(num_agents,num_vacnJobs,
    priorityWeightCoeff=1,affinityWeightCoeff=1,penalty1=2,penalty2=2,control_restrictions=True):
    if(control_restrictions and num_agents not in range(3,21)):
        raise ValueError('Not respecting the 3<= num_agents <=20 restriction')
    if(control_restrictions and num_vacnJobs not in range(5,31)):
        raise ValueError('Not respecting the 5<= num_vacnJobs <=30 restriction')

    priorities = [1,2,3,4,5]
    affinity_maximum = 0.5

    agents = list(range(num_agents))
    assgJobs = list(range(num_agents))
    vacnJobs = list(range(num_vacnJobs))

    agents_assgJobs = assgJobs.copy()
    r.shuffle(agents_assgJobs)
    
    agents_assgJobsAfinnity = []
    agents_vacnJobsAfinnity = []
    for agent in agents:
        agents_assgJobsAfinnity.append(
            [round(r.random() * affinity_maximum,2) for _ in range(num_agents)]
        )
        agents_vacnJobsAfinnity.append(
            [round(r.random() * affinity_maximum,2) for _ in range(num_vacnJobs)]
        )

    assgJobs_agents = [None] * len(agents_assgJobs)
    for i, value in enumerate(agents_assgJobs):
        assgJobs_agents[value] = i  
    
    assgJobs_priority = r.choices(priorities,weights=range(1, len(priorities) + 1), k=num_agents)
    vacnJobs_priority = r.choices(priorities,weights=range(1, len(priorities) + 1), k=num_vacnJobs)

    allBinaryVariables = list(range(num_vacnJobs * num_agents))

    data = {
        "num_agents": num_agents,
        "num_vacnJobs": num_vacnJobs,
        "priorityWeightCoeff": priorityWeightCoeff,
        "affinityWeightCoeff": affinityWeightCoeff,
        "penalty1": penalty1,
        "penalty2": penalty2,
        "agents": agents,
        "assgJobs": assgJobs,
        "vacnJobs": vacnJobs,
        "agents_assgJobs": agents_assgJobs,
        "agents_assgJobsAfinnity":agents_assgJobsAfinnity,
        "agents_vacnJobsAfinnity":agents_vacnJobsAfinnity,
        "assgJobs_agents": assgJobs_agents,
        "assgJobs_priority": assgJobs_priority,
        "vacnJobs_priority": vacnJobs_priority,
        "allBinaryVariables": allBinaryVariables
    }

    json_data = json.dumps(data, indent=4)
    return json_data

In [7]:
import json
import os

instances_indexes = range(5)
problem_size = [
    [3,5],
    [3,6],
    [4,5],
    [5,5],
    [6,5]
]

for index in instances_indexes:
    instance = create_random_instance(problem_size[index][0],problem_size[index][1])

    '''
    WARNING:
    The next piece of code can rewrite the alredy existing instances. If you are sure about doing this, uncomment the code and change the file name
    from 'instance%s_aux.json' to 'instance%s.json'.

    In case you want to generate new instances but not rewritting the alredy existing one, just leave the file name 'instance%s_aux.json'.
    '''
    with open('instance%s/instance%s.json'% (str(index),str(index)), 'w') as file:
        file.write(instance)
